## Exploring Equity Classifications with Machine Learning
### Proposal
DATE: May 20, 2021

TO: Annette Demchur, Rebecca Morgan

FROM: Margaret Atkinson

RE: Staff Initiated Study Proposal:

#### Exploring Equity Classifications with Machine Learning

I would like to conduct research to answer the following question: Could an
unsupervised machine learning algorithm create groups of towns based on
demographic information that would be useful to explore questions about equity? To
explain, when we compare all towns that pass the minority threshold (or TAZs or block
groups etc.) to all towns that do not, we may be missing the way the demographic
variables interact and a multifactored grouping could allow us to explore the
demographic towns with a more detailed approach without the blending nature of an
index.

This project would use python and specifically the Scikit-Learn python library to
conduct unsupervised machine learning based on demographic data at the town level.
The data would be demographic Census data from the American Community Survey 5-
year estimates at minimum on the topics of: Race/Ethnicity, Limited English Proficiency,
Median Income, Low Income, No Car Households, Population Density, Children, and
Seniors. The product would be a geographic file that shows groupings of towns by
demographic profile as found by the unsupervised machine learning algorithm as well
as a written description of what each grouping represents.

If the question is pursued and the results are useful - the ultimate intention (as a
follow up project) would be to look at the way MPO distributes funding between the
groups and within each group to look for disparities. Explanations of disparities could
lead to a re-examination of the variables used in the algorithm in order to provide
an additional check for equitable spending.


Towns in the MPO
BRMPO <- c("Beverly","Boston","Braintree","Cambridge","Chelsea","Everett","Framingham","Franklin","Gloucester","Lynn","Malden","Marlborough","Medford","Melrose","Newton","Peabody","Quincy","Revere","Salem","Somerville","Waltham","Watertown","Weymouth","Woburn","Acton","Arlington","Ashland","Bedford","Bellingham","Belmont","Bolton","Boxborough","Brookline","Burlington","Canton","Carlisle","Cohasset","Concord","Danvers","Dedham","Dover","Essex","Foxborough","Hamilton","Hingham","Holbrook","Holliston","Hopkinton","Hudson","Hull","Ipswich”,"Lexington","Lincoln","Littleton","Lynnfield","Manchester-by-the-Sea","Marblehead","Marshfield","Maynard","Medfield",           "Medway","Middleton","Milford","Millis","Milton","Nahant","Natick","Needham","Norfolk","North Reading","Norwell","Norwood","Randolph","Reading","Rockland","Rockport","Saugus","Scituate","Sharon","Sherborn","Southborough","Stoneham","Stow","Sudbury","Swampscott","Topsfield","Wakefield","Walpole","Wayland","Wellesley","Wenham","Weston","Westwood","Wilmington","Winchester","Winthrop","Wrentham")

Algorithms
K-means with K-means++ as initialization of seeding
Only deterministic if the seeds are always the same, which K-means++ does not guarantee.
DBSCAN


Spectral Clustering
Fully deterministic



| Demographic | Tables | Fields | Calculation | Notes |
| ----------- | ------ | ------ | ----------- | ----- |
| Race/Ethnicity | ACS14: B02001 | B02001\_001-B02001\_002 | Total - White Alone|  |
| Limited English Proficiency | ACS14: B16001 | B16001\_005 + B16001\_008 + B16001\_011 + B16001\_014 + B16001\_017 + B16001\_020 + B16001\_023 + B16001\_026 + B16001\_029 + B16001\_032 + B16001\_035 + B16001\_038 + B16001\_041 + B16001\_044 + B16001\_047 + B16001\_050 + B16001\_053 + B16001\_056 + B16001\_059 + B16001\_062 + B16001\_065 + B16001\_068 + B16001\_071 + B16001\_074 + B16001\_077 + B16001\_080 + B16001\_083 + B16001\_086 + B16001\_089 + B16001\_092 + B16001\_095 + B16001\_098 + B16001\_101 + B16001\_104 + B16001\_107 + B16001\_110 + B16001\_113 + B16001\_116 + B16001\_119 | C16001 (less than very well)/(Total Population - (B01001\_003 +B01001\_027) |     
|Median Income| ACS14: B19013 | B19013\_001||                                                                                    |Low Income Households| ACS14: B19001, B19025, B11001| All of B19001, B19025\_001, B11001\_001 | HH Income Ranges, Aggregate HH Income, Total HH | % of HH with income below 200% of poverty line|
| No Car Households| ACS14: B08201| B08201\_002| HH with no vehicles available||
| Population Density | https://jtleider.github.io/censusdata/api.html | B01001\_001/AREA | Total Pop / AREA | These are shape, also use total population data |
| Children | ACS14: B01001,  2010Cen: P12| (B01001\_003 + B01001\_004 + B01001\_005 + B01001\_006 + B01001\_027 + B01001\_028 + B01001\_029 + B01001\_030),  (P012\_003 + P012\_004 + P012\_005 + P012\_006 + P012\_027 + P012\_028 + P012\_029 + P012\_030) | Boys under 18 plus girls under 18 | 0-17 |
| Population Over 5 | ACS14: B01001,  2010Cen: P12  | B01001\_001 - (B01001\_003 +B01001\_027), P012\_001 - (P012\_003 + P012\_027)  | Total Population - Children under 5 | 5+ |
| Seniors | ACS14: B01001 | (B01001\_020 + B01001\_021 + B01001\_022 + B01001\_023 + B01001\_024 + B01001\_025) + (B01001\_044 + B01001\_045 + B01001\_046 + B01001\_047 + B01001\_048 + B01001\_049) | Men ages 65+ plus Women ages 65+ | 65+ |
| People with Disabilities | ACS14: S1810 | S1810_C02_001E / S1810\_C01\_001E ( total pop with disability / total non institutionalized population) | Includes: Ambulatory, Hearing, Vision, Self-Care, Cognitive, Independent Living Difficulties |
| Total Population | ACS14: B01001,  2010Cen: P1 | B01001\_001, P01\_001 | Includes those housed in group quarters |



In [11]:
import numpy as np
import pandas as pd
import geopandas as gpd
import censusdata
import matplotlib
import sklearn
from functools import reduce
#import plotly.express as px

In [2]:
#set some parameters to make display of data nicer
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [3]:
mpoTowns = ["Beverly","Boston","Braintree","Cambridge","Chelsea","Everett","Framingham","Franklin","Gloucester",
            "Lynn","Malden","Marlborough","Medford","Melrose","Newton","Peabody","Quincy","Revere","Salem","Somerville",
            "Waltham","Watertown","Weymouth","Woburn","Acton","Arlington","Ashland","Bedford","Bellingham","Belmont",
            "Bolton","Boxborough","Brookline","Burlington","Canton","Carlisle","Cohasset","Concord","Danvers","Dedham",
            "Dover","Essex","Foxborough","Hamilton","Hingham","Holbrook","Holliston","Hopkinton","Hudson","Hull","Ipswich",
            "Lexington","Lincoln","Littleton","Lynnfield","Manchester-by-the-Sea","Marblehead","Marshfield","Maynard",
            "Medfield", "Medway","Middleton","Milford","Millis","Milton","Nahant","Natick","Needham","Norfolk",
            "North Reading","Norwell","Norwood","Randolph","Reading","Rockland","Rockport","Saugus","Scituate",
            "Sharon","Sherborn","Southborough","Stoneham","Stow","Sudbury","Swampscott","Topsfield","Wakefield","Walpole",
            "Wayland","Wellesley","Wenham","Weston","Westwood","Wilmington","Winchester","Winthrop","Wrentham"]

mpoNums = ['09175','39765','41515','44105','50250','04930','17405','24820','14640','74175','16495','60785','41690','55745',
          '55955','01605','15060','43895','80510','11000','56130','62535','02130','07350','24925','73790','11525','31540',
          '39625','61380','67665','80230','09840','35425','68260','72215','81035','77255','05070','35215','68050','39835',
          '40115','72600','00380','37875','31085','06365','41165','56585','07000','13205','30700','05595','21850','37995',
          '43580','57880','40430','52490','07740','78972','81005','37560','68645','74595','59105','60015','38400','16250',
          '37490','70150','26150','27900','50145','38855','60330','30210','31645','25172','21990','48955','45560','57775',
          '32310','41095','73440','11315','46050','72495','39975','30455','82315','78690','35950','38715','63165']

In [4]:
censusdata.printtable(censusdata.censustable('acs5', 2014, 'B19013'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19013_001E  | MEDIAN HOUSEHOLD INCOME IN THE | !! Estimate Median household income in the past 12 month | int  
-------------------------------------------------------------------------------------------------------------------


In [5]:
#censusdata.geographies(censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]), 'acs5', 2014)

## Finding Tables and Fields Resources

https://www.census.gov/prod/cen2010/doc/sf1.pdf

https://www.census.gov/programs-surveys/acs/technical-documentation/table-shells.2014.html

In [21]:
#create lists of fields needed for each calculation
#ACS 2014 columns
race = ['B02001_001E','B02001_002E'] #universe = total population
lep = ['B16001_005E','B16001_008E','B16001_011E','B16001_014E','B16001_017E','B16001_020E','B16001_023E','B16001_026E',
       'B16001_029E','B16001_032E','B16001_035E','B16001_038E','B16001_041E','B16001_044E','B16001_047E','B16001_050E',
       'B16001_053E','B16001_056E','B16001_059E','B16001_062E','B16001_065E','B16001_068E','B16001_071E','B16001_074E',
       'B16001_077E','B16001_080E','B16001_083E','B16001_086E','B16001_089E','B16001_092E','B16001_095E','B16001_098E',
       'B16001_101E','B16001_104E','B16001_107E','B16001_110E','B16001_113E','B16001_116E','B16001_119E','B01001_001E',
       'B01001_003E','B01001_027E'] #universe = population > age 5
medinc = ['B19013_001E'] #No Universe needed, but HH
nocar = ['B08201_002E'] #Universe = Households
u18 = ['B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_027E','B01001_028E',
       'B01001_029E','B01001_030E'] #universe = total population
sen = ['B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E',
       'B01001_044E','B01001_045E','B01001_046E','B01001_047E','B01001_048E','B01001_049E']#universe = total population
dis = ['S1810_C02_001E','S1810_C01_001E'] #universe = total NON institutionalized population
acsHH = ['B11001_001E']
acsPlus5 = ['B01001_001E','B01001_003E', 'B01001_027E']
acsTotPop =  ['B01001_001E']

#decennial census columns
totpop = ['P001_001E']
plus5 = ['P012_0001E','P012_0003E','P012_0027E']
hh = ['P018_0001E']

### Grab ACS Data, Brief Clean, and Sum if Necessary

In [22]:
#################################GRAB ACS DATA###################################################
#MINORITY
#get table and reset index so that county sub is a field called muni
minority = censusdata.download('acs5',2014,
                    censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]),
                   race).reset_index().rename(columns={'index':'muni'})

#create the actual minority field
minority['Minority#'] = minority['B02001_001E'] - minority['B02001_002E']
#filter down to just what is in the MPO
minority=minority[minority.apply(lambda r: any([town in str(r[0]) for town in mpoNums]), axis=1)]


#LEP
limEngProf = censusdata.download('acs5',2014,
                    censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]),
                   lep).reset_index().rename(columns={'index':'muni'})

#create the actual LEP field
limEngProf['LEP#'] = limEngProf[list(limEngProf.columns)[:-1]].sum(axis=1)

#filter down to just what is in the MPO
limEngProf=limEngProf[limEngProf.apply(lambda r: any([town in str(r[0]) for town in mpoNums]), axis=1)]


#MEDIAN INCOME
medIncome = censusdata.download('acs5',2014,
                    censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]),
                   medinc).reset_index().rename(columns={'index':'muni', 'B19013_001E':'MedianIncome'})
#filter down to just what is in the MPO
medIncome=medIncome[medIncome.apply(lambda r: any([town in str(r[0]) for town in mpoNums]), axis=1)]


#NO CAR HOUSEHOLDS
noCarHH = censusdata.download('acs5',2014,
                    censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]),
                   nocar).reset_index().rename(columns={'index':'muni', 'B08201_002E':'NoCarHH#'})
#filter down to just what is in the MPO
noCarHH =noCarHH[noCarHH.apply(lambda r: any([town in str(r[0]) for town in mpoNums]), axis=1)]


#CHILDREN
kids = censusdata.download('acs5',2014,
                    censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]),
                   u18).reset_index().rename(columns={'index':'muni'})

#create the actual LEP field
kids['Under18#'] = kids[list(kids.columns)[:-1]].sum(axis=1)

#filter down to just what is in the MPO
kids=kids[kids.apply(lambda r: any([town in str(r[0]) for town in mpoNums]), axis=1)]


#SENIORS
seniors = censusdata.download('acs5',2014,
                    censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]),
                   sen).reset_index().rename(columns={'index':'muni'})

#create the actual LEP field
seniors['Over65#'] = seniors[list(seniors.columns)[:-1]].sum(axis=1)

#filter down to just what is in the MPO
seniors=seniors[seniors.apply(lambda r: any([town in str(r[0]) for town in mpoNums]), axis=1)]


#DISABILITY
disability = censusdata.download('acs5',2014,
                    censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]),
                 dis,'e4bec76221ba04c7df76c7c580659bf1f54ed2c1',
                                 'subject').reset_index().rename(columns={'index':'muni','S1810_C02_001E':'Disability#'})
#filter down to just what is in the MPO
disability=disability[disability.apply(lambda r: any([town in str(r[0]) for town in mpoNums]), axis=1)]
#universe in same table pretty much only so do %calc here
disability['Disability%'] = disability['Disability#']/disability['S1810_C01_001E']


#HOUSEHOLDS 
households = censusdata.download('acs5',2014,
                    censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]),
                   acsHH).reset_index().rename(columns={'index':'muni', 'B11001_001E':'Households#'})
#filter down to just what is in the MPO
households =households[households.apply(lambda r: any([town in str(r[0]) for town in mpoNums]), axis=1)]


#OVER FIVE
#CHILDREN
over5 = censusdata.download('acs5',2014,
                    censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]),
                   acsPlus5).reset_index().rename(columns={'index':'muni'})

#create the actual LEP field
over5['Over5#'] = over5['B01001_001E'] - (over5['B01001_003E'] +over5['B01001_027E'])

#filter down to just what is in the MPO
over5=over5[over5.apply(lambda r: any([town in str(r[0]) for town in mpoNums]), axis=1)]


#TOTAL POPULATION
acsPop = censusdata.download('acs5',2014,
                    censusdata.censusgeo([('state', '25'), ('county','017,021,025,009,023,027' ), ('county subdivision', '*')]),
                   acsTotPop).reset_index().rename(columns={'index':'muni', 'B01001_001E':'TotalPopulation'})
#filter down to just what is in the MPO
acsPop =acsPop[acsPop.apply(lambda r: any([town in str(r[0]) for town in mpoNums]), axis=1)]

In [23]:
#Merge ACS data into one dataframe and make %
#collect all the data (and filter)
acsdata = [minority[['muni', 'Minority#']], limEngProf[['muni', 'LEP#']], medIncome[['muni', 'MedianIncome']], 
           noCarHH[['muni', 'NoCarHH#']], kids[['muni', 'Under18#']], seniors[['muni', 'Over65#']], 
           disability[['muni', 'Disability#', 'Disability%']], households[['muni','Households#']], over5[['muni','Over5#']], 
           acsPop[['muni','TotalPopulation']]]
#actually do the merging
acsDF = reduce(lambda  left,right: pd.merge(left,right,on=['muni'], how='outer'), acsdata)

#make the % fields
acsDF['Minority%'] = acsDF['Minority#']/acsDF['TotalPopulation']
acsDF['LEP%'] = acsDF['LEP#']/acsDF['Over5#']
acsDF['noCarHH%'] = acsDF['NoCarHH#']/acsDF['Households#']
acsDF['Under18%'] = acsDF['Under18#']/acsDF['TotalPopulation']
acsDF['Over65%'] = acsDF['Over65#']/acsDF['TotalPopulation']


In [26]:
#bring in shapefile for area for population density
cousub= gpd.read_file(r'C:\Users\AtkinsonM\OneDrive - Commonwealth of Massachusetts\Documents\Jupyter_Home\tl_2010_25_cousub10')

#get area in sq miles (default to square meters because of CRS)
cousub['Area'] = cousub.area*0.00000038610215855
cousub

,STATEFP10,COUNTYFP10,COUSUBFP10,COUSUBNS10,GEOID10,NAME10,NAMELSAD10,LSAD10,CLASSFP10,MTFCC10,CNECTAFP10,NECTAFP10,NCTADVFP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,Area
0,25,005,20100,00619433,2500520100,Easton,Easton town,43,T1,G4040,715,71650,72104,A,74468685,1241437,+42.0385331,-071.1086177,"POLYGON ((-71.14226 42.07136, -71.14073 42.071...",3.18e-09
1,25,005,22130,00618281,2500522130,Fairhaven,Fairhaven town,43,T1,G4040,780,75550,None,A,31947040,4542710,+41.6375027,-070.8740086,"POLYGON ((-70.84482 41.64135, -70.84452 41.638...",1.52e-09
2,25,005,23000,00618282,2500523000,Fall River,Fall River city,25,C5,G4040,780,77200,None,F,85811381,18424717,+41.7253505,-071.0941619,"POLYGON ((-71.19826 41.67841, -71.19953 41.679...",4.35e-09
3,25,005,25240,00619434,2500525240,Freetown,Freetown town,43,T1,G4040,780,75550,None,A,89286664,4995598,+41.7793185,-071.0194863,"POLYGON ((-70.97986 41.72709, -70.98523 41.724...",3.94e-09
4,25,005,38225,00618283,2500538225,Mansfield,Mansfield town,43,T1,G4040,715,71650,71654,A,52040205,851508,+42.0172998,-071.2172998,"POLYGON ((-71.28694 42.01498, -71.28414 42.016...",2.22e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,25,027,40255,00618371,2502740255,Mendon,Mendon town,43,T1,G4040,715,71650,73104,A,46034725,552653,+42.0944698,-071.5449020,"POLYGON ((-71.49772 42.09456, -71.49772 42.094...",1.96e-09
353,25,027,41165,00618372,2502741165,Milford,Milford town,43,T1,G4040,715,71650,73104,A,38204407,711615,+42.1586917,-071.5214188,"POLYGON ((-71.50503 42.19001, -71.50476 42.190...",1.64e-09
354,25,027,41340,00618373,2502741340,Millbury,Millbury town,43,T1,G4040,715,79600,None,A,40684061,1884818,+42.1922464,-071.7776490,"POLYGON ((-71.80644 42.15505, -71.80906 42.153...",1.79e-09
355,25,027,41585,00618374,2502741585,Millville,Millville town,43,T1,G4040,780,77200,None,A,12706301,151536,+42.0366420,-071.5787791,"POLYGON ((-71.56808 42.06400, -71.56392 42.064...",5.40e-10


In [36]:
acsDF.muni.str.slice(-5, None)


nan

### Grab Census Data, Brief Clean

Notes:
1. Bring in Census Data
2. Create %
3. Create raw #'s by conforming to census data
4. Prep for ML
5. Histograms of variables
6. Elbow diagram